In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1OAK'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1OAK.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1OAK
outputs dir:  ../outputs/1O/1OAK
[{'chains': [{'AA residues': 327,
              'chain_id': 'L',
              'seq': 'DIQMTQSPSSLSASLGDRVTISCSASQDINKYLNWYQQKPDGAVKLLIFYTSSLHSGVPSRFSGSGSGTDYSLTISNLEPEDIATYYCQQYEKLPWTFGGGTKLEVKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRN'},
             {'AA residues': 355,
              'chain_id': 'H',
              'seq': 'QVQLAESGPGLVAPSQSLSITCTVSGFSLTDYGVDWVRQPPGKGLEWLGMIWGDGSTDYNSALKSRLSITKDNSKSQVFLKMNSLQTDDTARYYCVRDPADYGNYDYALDYWGQGTSVTVSSAKTTPPSVYPLAPGNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSTWPSETVTCNVAHPASSTKVDKKIVPRD'},
             {'AA residues': 310,
              'chain_id': 'A',
              'seq': 'MYCSRLLDLVFLLDGSSRLSEAEFEVLKAFVVDMMERLRISQKWVRVAVVEYHDGSHAYIGLKDRKRPSELRRIASQVKYAGSQVASTSEVLKYTLFQIFSKIDRPEASRIALLLMASQEPQRMSRNFVRYVQGLKKKKVIVIPVGIGPHANLKQIRLIEKQAPENKAFVLSSVDELEQQRDEIVSYLCDLAPEAP'}],
  'model_id': 0}]


In [6]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'nmc-4 igg1',
  'chain': 'l',
  'fragment': 'fab fragment, an anti von willebrand factor (vwf) a1domain ',
  'engineered': 'yes',
  'other_details': 'the antibody heavy chain consists of two segments, h1and h2, both labeled chain h in this entry '},
 '2': {'misc': '',
  'molecule': 'nmc-4 igg1',
  'chain': 'h',
  'fragment': 'fab fragment, an anti von willebrand factor (vwf) a1domain ',
  'engineered': 'yes',
  'other_details': 'the antibody heavy chain consists of two segments, h1and h2, both labeled chain h in this entry '},
 '3': {'misc': '',
  'molecule': 'von willebrand factor',
  'chain': 'a',
  'fragment': 'a1 domain residues 507 - 702, or glycoprotein iba',
  '(a\\': 'alpha) binding domain',
  'engineered': 'yes'}}

In [7]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.info)

Pairwise chains of 1OAK are updated.
1OAK
[{'A': 'antigen', 'L': 'single'}, {'A': 'antigen', 'H': 'single'}]


In [8]:
# split chains
p.chains_to_pdb()

Saved chain ['L'] to ../outputs/1O/1OAK/1OAK_L.pdb
Saved chain ['A'] to ../outputs/1O/1OAK/1OAK_A.pdb
Saved chain ['L', 'A'] to ../outputs/1O/1OAK/1OAK_L_A.pdb
Saved chain ['H'] to ../outputs/1O/1OAK/1OAK_H.pdb
Saved chain ['H', 'A'] to ../outputs/1O/1OAK/1OAK_H_A.pdb


In [9]:
p.bounded

[{'type': 'single-antigen',
  'chains': [['L'], ['A'], ['L', 'A']],
  'pdb_files': ['../outputs/1O/1OAK/1OAK_L.pdb',
   '../outputs/1O/1OAK/1OAK_A.pdb',
   '../outputs/1O/1OAK/1OAK_L_A.pdb']},
 {'type': 'single-antigen',
  'chains': [['H'], ['A'], ['H', 'A']],
  'pdb_files': ['../outputs/1O/1OAK/1OAK_H.pdb',
   '../outputs/1O/1OAK/1OAK_A.pdb',
   '../outputs/1O/1OAK/1OAK_H_A.pdb']}]

In [10]:
p.save_annot()

https://www.ebi.ac.uk/pdbe/api/mappings/1oak
Save annotation of 1OAK: ../outputs/1O/1OAK/meta.json


## SASA

In [11]:
# calculate SASA
sasa_file = p.build_freesasa()

In [12]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain L ~ antigen chain A
------------------------------
key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain H ~ antigen chain A
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [13]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/1O/1OAK/seq_sasa_delta-0_span-30.p


([('single',
   [{'seq': 'NKY',
     'sig_res': 2,
     'start': (29, '30'),
     'end': (31, '32'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YEKLPW',
     'sig_res': 4,
     'start': (90, '91'),
     'end': (95, '96'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'RNFVRYVQ',
     'sig_res': 4,
     'start': (125, '632'),
     'end': (132, '639'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}]),
  ('single',
   [{'seq': 'TDYGVDWVRQPPGKGLEWLGMIWGDGSTD',
     'sig_res': 9,
     'start': (29, '244'),
     'end': (57, '272'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'DPADYGNYDY',
     'sig_res': 6,
     'start': (97, '312'),
     'end': (106, '321'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'PQRMSRNFVRYVQGLKKKKVIVIPVGIGPHANLKQIRLIEKQ',
     'sig_res': 14,
     'start': (120, '627'),
     'end': (161, '668'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}])],
 '../output

In [14]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'RNFVRYVQ',
  'input_label': 'antigen',
  'output': 'NKY | YEKLPW',
  'output_label': 'single'},
 {'input': 'PQRMSRNFVRYVQGLKKKKVIVIPVGIGPHANLKQIRLIEKQ',
  'input_label': 'antigen',
  'output': 'TDYGVDWVRQPPGKGLEWLGMIWGDGSTD | DPADYGNYDY',
  'output_label': 'single'}]

In [15]:
# complex
p2.print_delta_sasa(0)

complex: single chain L ~ antigen chain A
------------------------------
single chain:
   chain_id res_no  res aa      value
29        L     30  ASN  N   8.129336
31        L     32  TYR  Y  34.581973
90        L     91  TYR  Y   1.998656
91        L     92  GLU  E  32.645169
93        L     94  LEU  L  35.150451
95        L     96  TRP  W  29.741885
------------------------------
antigen chain:
    chain_id res_no  res aa      value
125        A    632  ARG  R  68.824014
126        A    633  ASN  N   2.160662
129        A    636  ARG  R  96.332503
132        A    639  GLN  Q   1.998656


## distance

In [16]:
# calculate distance
dist_file = p.build_dist()

In [17]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
single chain L ~ antigen chain A
------------------------------
key1 ['type', 'chains', 'pdb_files', 'dist']
single chain H ~ antigen chain A
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [18]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('single',
   [{'seq': 'NKY',
     'sig_res': 2,
     'start': (29, 30),
     'end': (31, 32),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YEKLPW',
     'sig_res': 4,
     'start': (90, 91),
     'end': (95, 96),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'RNFVR',
     'sig_res': 2,
     'start': (125, 632),
     'end': (129, 636),
     'dist_cutoff': 5,
     'max_span': 30}]),
  ('single',
   [{'seq': 'MIWGDGSTD',
     'sig_res': 8,
     'start': (49, 264),
     'end': (57, 272),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'DPADYGNYDY',
     'sig_res': 6,
     'start': (97, 312),
     'end': (106, 321),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'EPQRMSRNFVRYVQGLKKKKVIVIPVGIGPHANLKQIRL',
     'sig_res': 14,
     'start': (119, 626),
     'end': (157, 664),
     'dist_cutoff': 5,
     'max_span': 30}])],
 '../outputs/1O/1OAK/seq_dist_dist-5_span-30.p')

In [19]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'RNFVR',
  'input_label': 'antigen',
  'output': 'NKY | YEKLPW',
  'output_label': 'single'},
 {'input': 'EPQRMSRNFVRYVQGLKKKKVIVIPVGIGPHANLKQIRL',
  'input_label': 'antigen',
  'output': 'MIWGDGSTD | DPADYGNYDY',
  'output_label': 'single'}]

In [20]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         1  ASP    C  12.735327  D
1         2  ILE    C   8.050718  I
2         3  GLN    C  13.890382  Q
3         4  MET    C  12.596424  M
4         5  THR    C  17.476336  T
..      ...  ...  ...        ... ..
207     208  SER    C  57.380472  S
208     209  PHE    C  56.356283  F
209     210  ASN    C  62.112721  N
210     211  ARG    C  61.351019  R
211     212  ASN    C  65.074196  N

[212 rows x 5 columns]
------------------------------
     res_no  res atom     value aa
125     632  ARG    C  3.639371  R
129     636  ARG    C  2.927584  R
